# Goal

Using ML algorithms such as transfer learing and Open CV Computer Vision to identify the objects in the video and build KPI metrics

## Steps:
1. Install and Import Dependencies
2. Load the model
3. Once we achieve the accuracy and performance - Create a docker which can be deployed to cloud platform - AWS, Azure or Google
4. Save it to the database
5. Create a dashboard with KPI metrics

## 1. Install and Import Dependencies

In [1]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

fatal: destination path 'yolov5' already exists and is not an empty directory.
/home/srinu/video-analytics/yolov5
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
from yolov5 import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v6.0-276-ga936f5f torch 1.10.2 CPU


Setup complete ✅ (12 CPUs, 62.5 GB RAM, 580.2/1828.6 GB disk)


In [3]:
from matplotlib import pyplot as plt
import numpy as np
import cv2

## 2. Load Model

In [4]:
# Detect only people
!python detect.py --weights yolov5s.pt --img 640 --conf 0.25 --classes 0 --source ../data/VIRAT_S_010002_01_000123_000148.mp4

detect: weights=['yolov5s.pt'], source=../data/VIRAT_S_010002_01_000123_000148.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=[0], agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.0-276-ga936f5f torch 1.10.2 CPU

100%|██████████████████████████████████████| 14.0M/14.0M [00:01<00:00, 11.0MB/s]

Fusing layers... 
Model Summary: 213 layers, 7225885 parameters, 0 gradients
video 1/1 (1/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 2 persons, Done. (0.065s)
video 1/1 (2/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 2 persons, Done. (0.041s)
video 1/1 (3/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 

video 1/1 (62/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 2 persons, Done. (0.046s)
video 1/1 (63/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 2 persons, Done. (0.045s)
video 1/1 (64/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 2 persons, Done. (0.046s)
video 1/1 (65/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 2 persons, Done. (0.044s)
video 1/1 (66/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 2 persons, Done. (0.047s)
video 1/1 (67/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 2 persons, Done. (0.047s)
video 1/1 (68/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 2 persons, Done. (0.051s)
video 1/1 (69/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 2 persons, Done. (0.044s)
video 1/1 (70/54

video 1/1 (129/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 Done. (0.042s)
video 1/1 (130/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 Done. (0.041s)
video 1/1 (131/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 Done. (0.044s)
video 1/1 (132/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 Done. (0.040s)
video 1/1 (133/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.041s)
video 1/1 (134/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.045s)
video 1/1 (135/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.043s)
video 1/1 (136/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.044s)
video 1/1 (137/548) /home/srinu/video-analytics/data/VIR

video 1/1 (198/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 Done. (0.044s)
video 1/1 (199/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.044s)
video 1/1 (200/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 Done. (0.045s)
video 1/1 (201/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 Done. (0.044s)
video 1/1 (202/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 Done. (0.047s)
video 1/1 (203/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 Done. (0.048s)
video 1/1 (204/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 2 persons, Done. (0.039s)
video 1/1 (205/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 2 persons, Done. (0.045s)
video 1/1 (206/548) /home/srinu/video-analytics/data/VIRAT_S_010

video 1/1 (268/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.044s)
video 1/1 (269/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.045s)
video 1/1 (270/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.046s)
video 1/1 (271/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 Done. (0.047s)
video 1/1 (272/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.045s)
video 1/1 (273/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 Done. (0.044s)
video 1/1 (274/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 Done. (0.046s)
video 1/1 (275/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.044s)
video 1/1 (276/548) /home/srinu/video-analytic

video 1/1 (338/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.046s)
video 1/1 (339/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.045s)
video 1/1 (340/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.047s)
video 1/1 (341/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.042s)
video 1/1 (342/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.043s)
video 1/1 (343/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.045s)
video 1/1 (344/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.046s)
video 1/1 (345/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.043s)
video 1/1 (346/5

video 1/1 (406/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 Done. (0.044s)
video 1/1 (407/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 Done. (0.047s)
video 1/1 (408/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.049s)
video 1/1 (409/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.047s)
video 1/1 (410/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.042s)
video 1/1 (411/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.043s)
video 1/1 (412/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.044s)
video 1/1 (413/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.045s)
video 1/1 (414/548) /home/srinu/vide

video 1/1 (474/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.041s)
video 1/1 (475/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.045s)
video 1/1 (476/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.043s)
video 1/1 (477/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.043s)
video 1/1 (478/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.045s)
video 1/1 (479/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.046s)
video 1/1 (480/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.044s)
video 1/1 (481/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.046s)
video 1/1 (482/5

video 1/1 (543/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 Done. (0.044s)
video 1/1 (544/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.044s)
video 1/1 (545/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 Done. (0.044s)
video 1/1 (546/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 1 person, Done. (0.041s)
video 1/1 (547/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 2 persons, Done. (0.044s)
video 1/1 (548/548) /home/srinu/video-analytics/data/VIRAT_S_010002_01_000123_000148.mp4: 384x640 2 persons, Done. (0.046s)
Speed: 0.3ms pre-process, 45.8ms inference, 1.5ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs/detect/exp


In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

In [ ]:
model

## 3. Make Detections

In [ ]:
img = 'https://media.istockphoto.com/photos/rushhour-traffic-in-downtown-beijing-picture-id911404660?b=1&k=20&m=911404660&s=170667a&w=0&h=yhxesKClntXOk70D3QGa5JWvTDp8keyG0NqOV6nJQx4='

In [ ]:
results = model(img)
results.print()

In [ ]:
%matplotlib inline
plt.imshow(np.squeeze(results.render()))
plt.show()

In [ ]:
#np.squeeze(results.render()).shape

## 4. Real Time Detections

In [ ]:
cap = cv2.VideoCapture('../data/VIRAT_S_010001_01_000071_000167.mp4')
while cap.isOpened():
    ret, frame = cap.read()
    
    # Make detections
    results = model(frame)
    
    cv2.imshow('YOLO', np.squeeze(results.render()))
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# 5. Train from scratch

In [ ]:
import uuid #unique identifier
import os
import Lime

In [ ]:
#model = torch.hub.load('ultralytics/yolov5', 'custom', path='../cfg/yolov3.weights', force_reload=True)